```{contents}
```
## RunnablePassthrough 


**RunnablePassthrough** is a utility in LangChain that **passes input data through a runnable chain unchanged** (or partially unchanged).
It is used to **fan out, preserve, or reattach original inputs** while other runnables transform data in parallel.

In short: **it forwards inputs as-is** so downstream components can still access them.

---

### Why RunnablePassthrough Is Needed

In modern LCEL (LangChain Expression Language) pipelines:

* Some steps **transform** data (retrievers, prompts, LLMs)
* Some steps still need **the original input**

RunnablePassthrough solves:

* Losing the original question after transformation
* Repeating expensive computations
* Manually wiring inputs

---

### Conceptual Flow

```
User Input
   ├── RunnablePassthrough → original input
   └── Other Runnable      → transformed output
                     ↓
                Combined Output
```

---

### Architecture View

![Image](https://cdn.analyticsvidhya.com/wp-content/uploads/2024/06/image-26-1.png)

![Image](https://miro.medium.com/1%2AX9hmWQYKaZevGO1SlGchGA.png)

![Image](https://dz2cdn1.dzone.com/storage/temp/18047285-screenshot-2024-11-18-at-121208pm.png)

---

### Basic Demonstration (Minimal)



In [1]:
from langchain_core.runnables import RunnablePassthrough

runnable = RunnablePassthrough()

runnable.invoke("Hello LangChain")


'Hello LangChain'



**Output**

```
Hello LangChain
```

No modification. Pure pass-through.

---

### RunnablePassthrough in a Chain

#### Problem Without Passthrough

You want:

* Original user question
* Retrieved context
* Final LLM answer

But retrieval **consumes** the question.

---

### Correct Pattern Using RunnablePassthrough



In [3]:
from langchain_core.runnables import RunnablePassthrough
from langchain_classic.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_template(
    """
    Answer the question using the context.

    Question: {question}
    Context: {context}
    """
)



---

### Parallel Execution with Passthrough



In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# Create sample documents
docs = [
    Document(
        page_content="Retrieval Augmented Generation (RAG) is a technique that combines retrieval with generation to provide better answers.",
        metadata={"source": "doc1"}
    ),
    Document(
        page_content="RAG retrieves relevant documents from a knowledge base and uses them as context for the LLM.",
        metadata={"source": "doc2"}
    ),
    Document(
        page_content="LangChain provides tools to build RAG pipelines efficiently.",
        metadata={"source": "doc3"}
    )
]

# Create embeddings and vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

# Create retriever
retriever = vectorstore.as_retriever()

In [6]:
chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever
    }
    | prompt
    | llm
)



**What happens**

* `RunnablePassthrough()` → forwards the original question
* `retriever` → transforms question → documents
* Both outputs are merged into a dict
* Prompt receives both fields

---

### End-to-End Example (RAG)



In [7]:
response = chain.invoke("What is retrieval augmented generation?")
print(response.content)


Retrieval Augmented Generation (RAG) is a technique that combines retrieval with generation to provide better answers. RAG retrieves relevant documents from a knowledge base and uses them as context for the LLM. LangChain provides tools to build RAG pipelines efficiently.




Here:

* Question flows untouched
* Context comes from retrieval
* LLM sees both

---

### RunnablePassthrough.assign() (Advanced)

Used to **add derived fields without losing input**.



In [12]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(
        length=lambda x: len(x["input"])
    )
)

chain.invoke({"input": "LangChain"})


{'input': 'LangChain', 'length': 9}



```python
```

**Output**

```python
{
  "input": "LangChain",
  "length": 9
}
```

---

### assign() in Real Pipelines



In [17]:
# Create a simple prompt for this example (without context requirement)
simple_prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question: {question}
    
    Metadata: {metadata}
    """
)

chain = (
    RunnablePassthrough.assign(
        metadata=lambda x: {"source": "user", "timestamp": "2024"}
    )
    | simple_prompt
    | llm
)

chain.invoke({"question":"What is LangChain?"})

AIMessage(content='LangChain is a hypothetical term and does not currently have a specific definition or known meaning in relation to any particular concept or technology. It could potentially refer to a blockchain platform or network designed for multilingual applications or services, but without further context or information, it is not possible to definitively define LangChain.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 36, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cpyky6ALZHgJqJ7EAnDBYIrgHW7Nr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b4bdc-a8e9-7f03-a838-feb4d5369be5-0', usage_



Useful for:

* Metadata injection
* Tracing
* Logging
* Guardrails

---

### Common Use Cases

* RAG pipelines
* Parallel runnable graphs
* Keeping user input alongside tool output
* Metadata enrichment
* Agent input preservation

---

### RunnablePassthrough vs lambda

| Aspect         | RunnablePassthrough | lambda  |
| -------------- | ------------------- | ------- |
| Readability    | High                | Low     |
| LCEL-native    | Yes                 | No      |
| Composable     | Yes                 | Limited |
| assign support | Yes                 | No      |

---

### When NOT to Use It

* When you actually want to transform input
* When a simple function is clearer
* When input preservation is unnecessary

---

### Key Takeaways

* RunnablePassthrough **forwards input unchanged**
* Essential for **parallel and fan-out pipelines**
* Prevents loss of original user input
* Core building block of **LCEL-style RAG**

